In [5]:
import matplotlib.pyplot as plt
import numpy as np
import random 

N = 50 

def pbc(i):
  """ Periodic boundary conditions; 
      In python one does not have to take care of i-1, when i=0. This is because of Python convention a[-1] = a[len(a)-1]  
  """ 
  if (i>N-1):
    return i-N
  return i 

def sign(x):
  if (x<0):
     return -1 
  return  1 

def magnetization(S):
  """ Total magnetization """
  return sum(S) 

def energy_density(S,h):
  """ Energy density """ 
  nearneigh = [S[i]*S[i+1] for i in range(N-1)]
  nearneigh = nearneigh + [S[N-1]*S[0]]
  mag_int = -h*magnetization(S)
  return (-sum(nearneigh)+mag_int)/N

def deltaE (S, i, h):
  return (2*S[i]*(S[i-1]+S[pbc(i+1)]) + 2 * h * S[i])

def mc_step(S, T, h):
  """ Performce one MC step; returns True if the trial configuration is accepted """ 
  site = random.randint(0, N-1)
  dE = deltaE (S, site, h)
  acc_prob  = min(np.exp()-dE/T,1)
  r = random.uniform(0,1)
  if (acc_prob>=r):
    S[site] = - S[site]
    return True
  return False 

def sweep(S, T, h):
  """ One sweep through the lattice -- usually N attempts to flip spins; here I use 2 N because I choose spins randomly """  
  for i in range(2*N):
    mc_step(S, T, h)

def skip(S, T, h, N=5):
  """ skip a few sweps for uncorrelated measurements """
  for i in range(N):
    sweep(S, T, h)

def correlation_func(x, N=20):
  k = np.array(range(N))
  C  = np.zeros(N) 
  y = x - sum(x)/len(x)
  for i in k:
    F = [ y[j]*y[j+i] for j in range(len(y)-N-1) ]
    C[i] = sum ( F ) #/ len(F)
  for i in range(1,N):
    C[i] = C[i]/C[0]
  C[0]=1
  return C

def average (x):
  return sum(x) / len(x)

def variance (x):
  return np.var(x) 

def bootstrap(r, f, M): 
  O = []
  for i in range(M):
    x = random.choices(r,k=len(r))
    O.append( f(np.array(x)) ) 
  
  mean = average(x) # ADD YOUR CODE HERE !!!!!!!!!!!!!!!!!!
  var = variance(x) # ADD YOUR CODE HERE  !!!!!!!!!!!!!!!!!! 
  return (mean, np.sqrt(var)) 


In [9]:
# Function to analyze thermalization: 
def thermalization(Temp, H):
  Scold =  np.zeros(N)+1
  Shot = np.array (  [sign(x) for x in np.random.uniform(-1,1,N) ] )

  Ecold = []
  Mcold = []
  Ehot = []
  Mhot = []

  for i in range(50000):
    Ecold.append(energy_density(Scold,H))
    Ehot.append(energy_density(Shot,H))
    Mcold.append(magnetization(Scold)/N)
    Mhot.append(magnetization(Shot)/N)
    
    mc_step(Scold, Temp, H)
    mc_step(Shot, Temp, H)

return (Ecold,Ehot,Mcold,Mhot)

SyntaxError: ignored

In [10]:
# Plot E for each MC step, using 2 initial conditions -- hot and cold starts.  
(Ec,Eh,Mc,Mh) = thermalization(3, 0)
plt.plot(Ec,'-')
plt.plot(Eh,'--')

plt.plot([0,200],[sum(Ec)/len(Ec)]*2,'k-')
plt.plot([0,200],[sum(Eh)/len(Eh)]*2,'k--')

plt.xlim(0,200)
plt.xlabel('Number of MC steps')
plt.ylabel(r'$E$')

ValueError: ignored

In [ ]:
# Plot C_E(k)
y = correlation_func(Ec, N=100)
plt.semilogy(np.abs(y),'.')
y = correlation_func(Eh, N=100)
plt.semilogy(np.abs(y),'.')
plt.xlabel('$k$')
plt.ylabel(r'$C(k)$')

In [ ]:
def ising_obs(Temp,H): 
  Scold =  np.zeros(N)+1
  Shot = np.array (  [sign(x) for x in np.random.uniform(-1,1,N) ] )
  
  Ecold = []
  Mcold = []
  Ehot = []
  Mhot = []

  ### add more observables to store below: 

  ###
  
  for i in range(30):   ### Thermalization --  to guranteee thermalization  
    sweep(Scold,Temp,H)
    sweep(Shot,Temp,H)


  for i in range(100):
    Ecold.append(energy_density(Scold,H))
    Ehot.append(energy_density(Shot,H))
    Mcold.append(magnetization(Scold)/N)
    Mhot.append(magnetization(Shot)/N)
    sweep(Scold,Temp,H)
    skip(Scold,Temp,H)
    sweep(Shot,Temp,H)
    skip(Shot,Temp,H)

  return (Ecold,Ehot,Mcold,Mhot)

In [ ]:
# Temperature dependence for <E> and <E^2> - <E>^2
arEc=[]
arEh=[]

arvEc=[]
arvEh=[]

arEc_err=[]
arEh_err=[]

arvEc_err=[]
arvEh_err=[]

temps = np.arange(1e-1,3,0.1)
for temp in temps:
  (Ec,Eh,Mc,Mh) = ising_obs(temp,0)
  aEc,aEc_err = bootstrap(Ec, average, 20)
  aEh,aEh_err = bootstrap(Eh, average, 20)
  arEc.append(aEc)
  arEc_err.append(aEc_err)
  arEh.append(aEh)
  arEh_err.append(aEh_err)

  vEc,vEc_err = bootstrap(Ec, variance, 20)
  vEh,vEh_err = bootstrap(Eh, variance, 20)
  arvEc.append(vEc)
  arvEc_err.append(vEc_err)
  arvEh.append(vEh)
  arvEh_err.append(vEh_err)

In [ ]:
x_analytic = np.linspace(1e-4,3,100)
y_analytic = -np.tanh(1/x_analytic)

plt.errorbar(temps,arEc,yerr=arEc_err,fmt='o')
plt.errorbar(temps,arEh,yerr=arEh_err,fmt='o')

plt.plot(x_analytic,y_analytic)

plt.xlabel('T')
plt.ylabel(r'$\langle E \rangle$')

In [ ]:
x_analytic = np.linspace(1e-4,3,100)
y_analytic = 1/N*1/np.cosh(1/x_analytic)**2 

plt.errorbar(temps,arvEc,yerr=arvEc_err,fmt='o')
plt.errorbar(temps,arvEh,yerr=arvEh_err,fmt='o')

plt.plot(x_analytic,y_analytic)
plt.xlabel('T')
plt.ylabel(r'$\langle E^2 \rangle - \langle E \rangle^2$')

# Plot $<E>$, $<M>$, $<M^2> - <M>^2$, $<E^2> - <E>^2$ dependence 
* on T for h = 0.1 and -0.1 
* on h for T = 0.1; 0.5; 1; 1.5; 2. 

Use bootstrap for the error estimate as done above. 

In [ ]:
\

# Spin-spin correlation 

In [ ]:
def C_spins(Temp,H): 
  Scold =  np.zeros(N)+1
  Shot = np.array (  [sign(x) for x in np.random.uniform(-1,1,N) ] )
  
  Cc = []
  Ch = []
  
  
  for i in range(30):   ### Thermalization --  to guranteee thermalization  
    sweep(Scold,Temp,H)
    sweep(Shot,Temp,H)

  for i in range(1000):
    C = np.zeros(int(N/2))

    for k in range(int(N/2)):
      C[k] = sum (  [Scold[j]*Scold[j+k]  for j in range(N-int(N/2)-1) ] )
    for k in range(1, int(N/2)):
      C[k] = C[k]/C[0]
    C[0] = 1 
    Cc.append(C)

    for k in range(int(N/2)):
      C[k] = sum (  [Shot[j]*Shot[j+k]  for j in range(N-int(N/2)-1) ] )
    for k in range(1, int(N/2)):
      C[k] = C[k]/C[0]
    C[0] = 1
    Ch.append(C)

    sweep(Scold,Temp,H)
    skip(Scold,Temp,H)
    sweep(Shot,Temp,H)
    skip(Shot,Temp,H)

  return (Cc,Ch)

In [ ]:
Cc,Ch =  C_spins(3,0) # T=3 

Cc_av,CC_err = bootstrap(Cc, average, 20)
Ch_av,Ch_err = bootstrap(Ch, average, 20)

plt.errorbar(range(len(Cc_av)),np.abs(Cc_av),yerr=CC_err,fmt='o')
plt.errorbar(range(len(Ch_av)),np.abs(Ch_av),yerr=Ch_err,fmt='o')
plt.yscale('log')

plt.xlabel(r'$k$')
plt.ylabel(r'$C(k)$')

plt.xlim(0,25)
plt.ylim(1e-3,2)

In [ ]:
# Plot the correlations funciton for T=2,1,0.5; 